# Transcript

In [1]:
import pandas as pd
transcript = pd.read_csv("../Data/transcript1.csv",index_col=0)
transcript

FileNotFoundError: [Errno 2] No such file or directory: '../Data/transcript1.csv'

In [ ]:
transcript.isnull().sum()

In [ ]:
transcript.dropna(inplace=True)
transcript

In [ ]:
transcript['event'].value_counts()

In [ ]:
transcript=transcript[transcript['event']!='transaction']

In [ ]:
transcript.rename(columns={"id":"person id"}, inplace=True)
transcript

# Profile

In [ ]:
profile = pd.read_csv("../Data/profile.csv",index_col=0)
profile.rename(columns={"id":"person id"}, inplace=True)
profile.head()

In [ ]:
profile.isnull().sum()

In [ ]:
profile.dropna(inplace=True)
profile.isnull().sum()

# Porfolio

In [ ]:
portfolio = pd.read_csv("../Data/portfolio.csv", index_col=0)
portfolio.drop(columns = ['reward', 'difficulty', 'duration','channels'], inplace = True)
portfolio

In [ ]:
portfolio.rename(columns = {'id':'offer id'}, inplace=True)
portfolio

# Merge Datasets

In [ ]:
transcript.shape

In [ ]:
profile.shape
profile

In [ ]:
df = pd.merge(transcript,profile,on='person id')
df

In [ ]:
portfolio

In [ ]:
df = pd.merge(df,portfolio, on='offer id')
df

## Dummy variables for gender and offer type

In [ ]:
#dummies for gender
df['female'] = [1 if i=='F' else 0 for i in df['gender']]
df

In [ ]:
#dummies for offer type
df['bogo'] = [1 if i=='bogo' else 0 for i in df['offer_type']]
df['informational'] =[1 if i=='informational' else 0 for i in df['offer_type']]
df

# Dummies for event

In [ ]:
#filter for completed ones and non-completed ones
red=df[df['event']=='offer completed']
no_red=df[df['event']!='offer completed']

In [ ]:
red

In [ ]:
#unique customer for redeemed 
temp=red.groupby('person id')['event'].count()
temp=temp.reset_index()
lst=red.columns[2:]
for col in lst:
    dic={}
    for i in temp['person id']:
        if i not in dic:
            dic[i]=red[red['person id']==i][col].iloc[0]
    temp[col]=temp['person id'].map(dic)

In [ ]:
temp

In [ ]:
#make sure it's done right. 4024 unique customers who redeemed offers
temp['person id'].value_counts()

In [ ]:
#unique customer for not redeemed 
temp2=no_red.groupby('person id')['event'].count()
temp2=temp2.reset_index()
lst2=no_red.columns[2:]
for col in lst2:
    dic2={}
    for i in temp2['person id']:
        if i not in (dic and dic2) : #the previous redeemed customer dictionary
            dic2[i]=no_red[no_red['person id']==i][col].iloc[0]
    temp2[col]=temp2['person id'].map(dic2)

In [ ]:
#dummies for event. 0 means not redeemed
temp2['event']=[0 for i in temp2['event']]

In [ ]:
temp2

In [ ]:
#check to see done correctly or not 
temp2[temp2['person id']=='78afa995795e4d85b5d9ceeca43f5fef']

In [ ]:
#put the two together 
newdf=pd.concat([temp, temp2], axis=0)

In [ ]:
newdf


# Create member since column

In [ ]:
# create member since column
newdf['became_member_on'] = pd.to_datetime(newdf['became_member_on'], format='%Y%m%d')
newdf['member_since'] = (pd.to_datetime('2021-01-01') - newdf['became_member_on']).astype('timedelta64[D]').astype(int)

In [ ]:
newdf